In [42]:
import sys
import os

dir_path = os.path.abspath("..")

if dir_path not in sys.path:
    sys.path.append(dir_path)

#sys.path.append(os.path.dirname(os.path.realpath(__file__)) + "/..")
import bar_chart_race as bcr
import pandas as pd
import warnings
from db.models import GameType, Player, SM5Game, Events, EventType, Team
from helpers import userhelper
from typing import List
from openskill.models.weng_lin.plackett_luce import PlackettLuceRating as Rating, PlackettLuce
import sanic
from tortoise import Tortoise
from config import config

warnings.filterwarnings("ignore")

model = PlackettLuce()

await Tortoise.init(
    db_url=f"mysql://{config['db_user']}:{config['db_password']}@{config['db_host']}:{config['db_port']}/laserforce",
    modules={"models": ["db.models"]}
)

data = {}
players_elo = {}

all_players = await Player.all()

for player in all_players:
    data[player.codename] = []
    players_elo[player.codename] = Rating(25, 25/3)

async def update_game_elo(game: SM5Game) -> bool:
    if not game.ranked:
        return False

    # go through all events for each game

    events: List[Events] = await game.events.filter(type__in=
        [EventType.DAMAGED_OPPONENT, EventType.DOWNED_OPPONENT, EventType.MISSILE_DAMAGE_OPPONENT,
        EventType.MISSILE_DOWN_OPPONENT, EventType.RESUPPLY_LIVES, EventType.RESUPPLY_AMMO]
    ).order_by("time").all() # only get the events that we need

    for event in events:
        if "@" in event.arguments[0] or "@" in event.arguments[2]:
            continue

        match event.type:
            case EventType.DAMAGED_OPPONENT | EventType.DOWNED_OPPONENT:
                shooter = await userhelper.player_from_token(game, event.arguments[0])
                shooter_player = await Player.filter(entity_id=shooter.entity_id).first()
                shooter_elo = players_elo[shooter_player.codename]
                target = await userhelper.player_from_token(game, event.arguments[2])
                target_player = await Player.filter(entity_id=target.entity_id).first()
                target_elo = players_elo[target_player.codename]

                out = model.rate([[shooter_elo], [target_elo]], ranks=[0, 1])

                players_elo[shooter_player.codename] = Rating(out[0][0].mu, shooter_player.sm5_sigma + ((out[0][0].sigma - shooter_player.sm5_sigma) * 0.1))
                players_elo[target_player.codename] = Rating(out[1][0].mu, target_player.sm5_sigma + ((out[1][0].sigma - shooter_player.sm5_sigma) * 0.1))

            case EventType.MISSILE_DAMAGE_OPPONENT | EventType.MISSILE_DOWN_OPPONENT:
                shooter = await userhelper.player_from_token(game, event.arguments[0])
                shooter_player = await Player.filter(entity_id=shooter.entity_id).first()
                shooter_elo = players_elo[shooter_player.codename]
                target = await userhelper.player_from_token(game, event.arguments[2])
                target_player = await Player.filter(entity_id=target.entity_id).first()
                target_elo = players_elo[target_player.codename]

                out = model.rate([[shooter_elo], [target_elo]], ranks=[0, 1])

                players_elo[shooter_player.codename] = Rating(out[0][0].mu, shooter_player.sm5_sigma + ((out[0][0].sigma - shooter_player.sm5_sigma) * 0.1))
                players_elo[target_player.codename] = Rating(out[1][0].mu, target_player.sm5_sigma + ((out[1][0].sigma - shooter_player.sm5_sigma) * 0.1))
    
    # rate game

    players_codenames = list(map(lambda x: x.name, await game.entity_starts.all()))

    team1 = []
    team2 = []

    for player in await game.entity_starts.filter(type="player"):
        if (await player.team).color_name == "Fire":
            team1.append(await Player.filter(entity_id=player.entity_id).first())
        else:
            team2.append(await Player.filter(entity_id=player.entity_id).first())

    team1_elo = list(map(lambda x: players_elo[x.codename], team1))
    team2_elo = list(map(lambda x: players_elo[x.codename], team2))

    if game.winner == Team.RED:
        team1_new, team2_new = model.rate([team1_elo, team2_elo], ranks=[0, 1])
    else:
        team1_new, team2_new = model.rate([team1_elo, team2_elo], ranks=[1, 0])

    for player, rating in zip(team1, team1_new):
        player.sm5_mu += (rating.mu - player.sm5_mu) * 5
        player.sm5_sigma += (rating.sigma - player.sm5_sigma) * 5

        players_elo[player.codename] = rating
    
    for player, rating in zip(team2, team2_new):
        player.sm5_mu += (rating.mu - player.sm5_mu) * 5
        player.sm5_sigma += (rating.sigma - player.sm5_sigma) * 5

        players_elo[player.codename] = rating

    for p_codename in data.keys():
        if p_codename in players_codenames:
            data[p_codename].append(players_elo[p_codename].mu - 3 * players_elo[p_codename].sigma)
        else:
            if len(data[p_codename]) == 0:
                data[p_codename].append(0)
            else:
                data[p_codename].append(data[p_codename][-1])

    return True

In [45]:
from IPython.display import clear_output
# update elo and format data
games = await SM5Game.all(ranked=False)
i = 0

for game in games:
    print(f"Updating elo for game {i}/{len(games)}")
    clear_output(wait=True)
    await update_game_elo(game)
    i += 1

data = {key:val for key, val in data.items() if val.count(0) != len(val)}

print(data)

df = pd.DataFrame(data=data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
{'SupremeKev': [0, 20.146839479061512, 21.475312769256988, 17.093399201351673, 17.093399201351673, 17.093399201351673, 17.093399201351673, 17.093399201351673, 19.444654539120037, 19.444654539120037, 18.41204160262166, 18.41204160262166, 18.41204160262166, 21.509104096356765, 21.509104096356765, 16.548488397696122, 16.548488397696122, 17.3178495454493, 21.519410360817762, 17.619488365501375, 17.619488365501375, 17.619488365501375, 17.619488365501375, 17.619488365501375, 20.117922753129623, 20.117922753129623, 18.723237836541728, 18.723237836541728, 18.723237836541728, 21.736510393488814, 21.736510393488814, 16.60422091530318, 16.60422091530318, 16.60422091530318, 16.60422091530318, 16.60422091530318, 16.60422091530318, 16.60422091530318, 21.852265864620236, 15.15103657820119, 20.326601030010607, 13.47092008659578, 13.47092008659578, 13.47092008659578, 13.

In [46]:
# show graph
bcr.bar_chart_race(df=df, filename=None, title="SM5 players at ILT skill rating over time", n_bars=20, period_fmt="Game {x:.0f}", period_length=750)